In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction\\exp'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction'

In [4]:
from mlproject.constants import *

[2025-05-10 22:37:49,653 : INFO : __init__ : Logger has been set up successfully!]


In [5]:
from dataclasses import dataclass
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
import json
import mlflow
import dagshub
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [6]:
from dataclasses import dataclass
import pandas as pd
import joblib
import json
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json
from mlproject import logger

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    metric_file_path: Path
    preprocessor_path: Path
    test_raw_data: Path  
    target_column: str
    all_params: dict

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LGBMClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            test_raw_data=config.test_raw_data,  
            target_column=schema.name,
            all_params=params
        )
        return model_evaluation_config

In [ ]:
import os
import json
import joblib
import mlflow
import dagshub
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlflow.models.signature import infer_signature
from mlproject import logger  
from mlproject.entities.config_entity import ModelEvaluationConfig  

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "39af2ec9240d8439ca7a568d37e4c8566f0e4507"

        dagshub.init(
            repo_owner="JavithNaseem-J",
            repo_name="Telecom-Customer-Churn-Prediction"
        )
        mlflow.set_tracking_uri(
            "https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow"
        )
        mlflow.set_experiment("Telecom-Customer-Churn-Prediction")

    def evaluate(self):
            mlflow.lightgbm.autolog()

            if not Path(self.config.test_raw_data).exists():
                raise FileNotFoundError(f"Test data not found at {self.config.test_raw_data}")
            if not Path(self.config.preprocessor_path).exists():
                raise FileNotFoundError(f"Preprocessor not found at {self.config.preprocessor_path}")
            if not Path(self.config.model_path).exists():
                raise FileNotFoundError(f"Model not found at {self.config.model_path}")

            with mlflow.start_run():
                mlflow.set_tag("model_type", "CatBoostClassifier")
                mlflow.set_tag("evaluation_stage", "testing")

                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)


                # Load and prepare test data
                logger.info(f"Loading test data from {self.config.test_raw_data}...")
                test_data = pd.read_csv(self.config.test_raw_data)
                target_column = self.config.target_column

                if target_column not in test_data.columns:
                    raise KeyError(f"Target column '{target_column}' not found in test data.")

                test_y = test_data[target_column]
                test_x = test_data.drop(columns=[target_column])
                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                logger.info("Making predictions...")
                predictions = model.predict(test_x_transformed)

                logger.info("Evaluating model performance...")

                precision = precision_score(test_y, predictions, average="weighted")

                recall = recall_score(test_y, predictions, average="weighted")
                
                f1 = f1_score(test_y, predictions, average="weighted")

                metrics = {
                    "accuracy": accuracy_score(test_y, predictions),
                    "precision": precision,
                    "recall": recall,
                    "f1": f1
                }

                mlflow.log_metrics(metrics)

                signature = infer_signature(test_x_transformed, predictions)
                mlflow.sklearn.log_model(
                    model,
                    artifact_path="TelecomCustomerChurnModel",
                    signature=signature,
                    registered_model_name="TelecomCustomerChurnModel"
                )

                logger.info(f"Evaluation Metrics:\n{json.dumps(metrics, indent=2)}")
                metrics_file = Path(self.config.root_dir) / "metrics.json"
                with open(metrics_file, "w") as f:
                    json.dump(metrics, f, indent=4)
                logger.info(f"Metrics saved to {metrics_file}")

                return metrics

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-05-10 22:38:13,992 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-05-10 22:38:14,003 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-05-10 22:38:14,019 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-05-10 22:38:14,020 : INFO : common : created directory at: artifacts]
[2025-05-10 22:38:14,021 : INFO : common : created directory at: artifacts/model_evaluation]


Repository initialized!

[2025-05-10 22:38:26,496 : INFO : 638826335 : Loading preprocessor and model...]
[2025-05-10 22:38:26,557 : INFO : 638826335 : Loading test data from artifacts/data_transformation/test.csv...]
[2025-05-10 22:38:26,585 : INFO : 638826335 : Test data shape: X=(2587, 19), y=(2587,)]
[2025-05-10 22:38:26,587 : INFO : 638826335 : Preprocessing test features...]
[2025-05-10 22:38:26,592 : INFO : 638826335 : Making predictions...]
[2025-05-10 22:38:31,935 : INFO : 638826335 : Evaluating model performance...]


Registered model 'TelecomCustomerChurnModel' already exists. Creating a new version of this model...
2025/05/10 22:38:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: TelecomCustomerChurnModel, version 5


[2025-05-10 22:38:52,004 : INFO : 638826335 : Evaluation Metrics:
{
  "accuracy": 0.8430614611519134,
  "precision": 0.8431700065577651,
  "recall": 0.8430614611519134,
  "f1": 0.843054754376749
}]
[2025-05-10 22:38:52,007 : INFO : 638826335 : Metrics saved to artifacts\model_evaluation\metrics.json]


Created version '5' of model 'TelecomCustomerChurnModel'.


🏃 View run able-foal-220 at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/0/runs/6796d6703c5948bda35e828b858b5941
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/0
